# ガバナンスシートを登録する
研究データを検証するための指針となるガバナンスシートを登録するタスクです。

## ガバナンスシート登録フォームを表示する
ガバナンスシートを登録するためのフォームを表示します。<br>
登録済みのガバナンスシートが存在する場合は呼び出して表示します。<br>
ガバナンスシートに設定した内容が研究データの検証の指針となります。<br>
ガバナンスシートは検証を実施した単位で保存されますので、どのタイミングでどのガバナンスシートで検証を行ったかは振り返ることができます。<br>
研究の進捗や公開の範囲に合わせて設定内容を編集してください。<br>


In [ ]:
# ガバナンスシート登録フォームを表示する
import json
import os
import traceback
from typing import Optional

from IPython.core.display import Javascript
from IPython.display import display, clear_output
import panel as pn
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils import dg_web
from library.utils.config import connect as con_config
from library.utils.config import message as msg_config
from library.utils.error import (UnusableVault, ProjectNotExist,
                                    UnauthorizedError, RepoPermissionError)
from library.utils.file import JsonFile
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import Button


notebook_name = 'governance_sheet.ipynb'


class GovSheet(TaskDirector):
    """ガバナンスシートの登録を行うクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox) : フォームを格納する。
            _msg_output (MessageBox) : ユーザーに提示するメッセージを格納する。
            schema(dict):取得したjsonschemaの値
            schema_form (Form) : フォームを作成する。
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            remote_path(str):リモート先のパス
            submit_button_title(str):ボタンのメッセージ
            submit_button(Button):ボタンの設定
            dg_web_url(str):DG-webのURL
            grdm_url(str):GRDMのURL
            grdm(Grdm):grdmファイルのGrdmクラス
    """

    def __init__(self, working_path: str) -> None:
        """GovSheet コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        self.schema_form = dg_web.Form()

        self.jsonfile_path = os.path.join(
            self._abs_root_path, 'data_governance/library/data/governance_sheet_order.json'
        )

        # フォームボックス
        self._form_box = self.schema_form.form_box
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = self.schema_form.msg_output
        self._msg_output.width = 900
        self.dg_web_url = con_config.get('DG_WEB', 'BASE_URL')
        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。
        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            self.remote_path = con_config.get('DG_WEB', 'GOVSHEET_PATH')
            self.schema = None
            clear_output()

            # データの取得
            if self.token and not self._msg_output.has_message():
                data = self.get_data(
                    token=self.token,
                    project_id=self.project_id, remote_path=self.remote_path
                )
                self.schema = self.get_schema()

                sorted_schema = self.schema_form.sort_order(self.schema, self.jsonfile_path)

            # フォーム定義
            if self.schema and not self._msg_output.has_message():
                self.schema_form.create_widgets(sorted_schema, data)
                self.submit_button_title = msg_config.get('save', 'submit')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self._form_box.append(self.submit_button)
                # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
                self.done_task()
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_schema(self) -> dict:
        """jsonschemaを取得するメソッドです。

        Returns:
            dict:jsonschemaの値を返す。
        """
        schema = {}
        try:
            dgwebApi = dg_web.Api()
            schema = dgwebApi.get_govsheet_schema(self.dg_web_url)
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return schema

    def get_data(self, token: str, project_id: str, remote_path: str) -> Optional[dict]:
        """指定したファイルの内容を取得するメソッドです。

        Args:
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            remote_path(str):リモート先のパス

        Returns:
            Optional[dict]:ファイルの内容の値を返す。
        """
        data = None
        try:
            data = self.grdm.download_json_file(
                token=token, base_url=self.grdm_url,
                project_id=project_id, remote_path=remote_path
            )
        except FileNotFoundError:
            data = None
        except json.JSONDecodeError:
            data = {}
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return data

    @TaskDirector.callback_form('ガバナンスシートを保存する')
    def submit(self, event):
        """ガバナンスシートを保存するメソッドです。"""
        file_path = os.path.join(self._abs_root_path, self.remote_path)
        tmpfile = JsonFile(file_path)
        try:
            data = self.schema_form.get_data()
            tmpfile.write(data)
            self.submit_button.disabled = True
            self.submit_button.set_looks_processing(msg_config.get('save', 'doing'))
            self.grdm.sync(
                    token=self.token,
                    base_url=self.grdm_url,
                    project_id=self.project_id,
                    abs_source=file_path,
                    abs_root=self._abs_root_path
                )
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
            return
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
            return
        finally:
            tmpfile.remove(missing_ok=True)
            self.submit_button.disabled = False
            self.submit_button.set_looks_init(self.submit_button_title)

        self._form_box.clear()
        self._msg_output.update_success(msg_config.get('dg_web', 'saved_govsheet'))
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()


GovSheet(os.path.abspath('__file__')).generate_form_section()

## カスタムガバナンスシート登録フォームを表示する
カスタムガバナンスシートを登録するフォームを表示します。

In [ ]:
# カスタムガバナンスシート登録フォームを表示する
import json
import os
import traceback

from IPython.core.display import Javascript
from IPython.display import clear_output, display
import panel as pn
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils.config import connect as con_config, message as msg_config, path_config
from library.utils.dg_web import Form
from library.utils.error import UnusableVault, RepoPermissionError, ProjectNotExist, UnauthorizedError
from library.utils.file import JsonFile
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import Button
from library.main_menu.subflow_controller import utils

notebook_name = 'governance_sheet.ipynb'

class CustomGovSheet(TaskDirector):
    """カスタムガバナンスシートの登録を行うクラスです。

    Attributes:
        instance:
            custom_form(Form): フォームを作成する。
            form_box(pn.WidgetBox): フォームを格納する。
            msg_output(MessageBox): ユーザーに提示するメッセージを格納する。
            grdm_url(str): GRDMのURL
            grdm(Grdm): grdmファイルのGrdmクラス
            token(str): パーソナルアクセストークン
            project_id(str): プロジェクトID
            submit_button_title(str): ボタンのメッセージ
            submit_button(Button): ボタンの設定
    """
    def __init__(self, working_path: str) -> None:
        """CustomGovSheetコンストラクタのメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        self.custom_form = Form()

        # フォームボックス
        self.form_box = self.custom_form.form_box
        self.form_box.width = 900
        # メッセージ用ボックス
        self.msg_output = self.custom_form.msg_output
        self.msg_output.width = 900

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            tuple[str, str]: GRDMのトークンの値とプロジェクトIDの値を返す。
        """
        token = ""
        project_id = ""

        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self.msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self.msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self.msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("2")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # サブフローステータス管理JSONの更新
        self.doing_task()
        # カスタムガバナンスシート、スキーマの取得
        custom_govsheet = utils.get_custom_govsheet(self._abs_root_path)
        custom_govsheet_schema = self.get_custom_govsheet_schema()

        if not custom_govsheet_schema:
            self.msg_output.update_info(msg_config.get('custom_govsheet', 'no_register_custom_govsheet'))

        try:
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            clear_output()

            if self.token and not self.msg_output.has_message():
                self.custom_form.create_widgets(custom_govsheet_schema, custom_govsheet)
                self.submit_button_title = msg_config.get('save', 'submit')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self.form_box.append(self.submit_button)
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)

        pn.extension()
        form = pn.WidgetBox()
        form.append(self.form_box)
        form.append(self.msg_output)
        display(form)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    @TaskDirector.task_cell("カスタムガバナンスシートを保存する")
    def submit(self, event):
        """カスタムガバナンスシートを保存するメソッドです。"""
        custom_govsheet_path = os.path.join(
            self._abs_root_path,
            path_config.DG_RESEARCHFLOW_FOLDER,
            'custom-gov-sheet.json'
        )
        try:
            data = self.custom_form.get_data()
            JsonFile(custom_govsheet_path).write(data)
            self.submit_button.disabled = True
            self.submit_button.set_looks_processing(msg_config.get('save', 'doing'))
            self.grdm.sync(
                self.token,
                self.grdm_url,
                self.project_id,
                custom_govsheet_path,
                self._abs_root_path
            )
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self.msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
            return
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)
            return
        finally:
            self.submit_button.disabled = False
            self.submit_button.set_looks_init(self.submit_button_title)

        self.form_box.clear()
        self.msg_output.update_success(msg_config.get('custom_govsheet', 'saved_custom_govsheet'))
        self.done_task()

    def get_custom_govsheet_schema(self) -> dict:
        """カスタムガバナンスシートのスキーマを取得するメソッドです。

        Returns:
            dict: カスタムガバナンスシートのスキーマ
        """
        custom_govsheet_schema_path = os.path.join(
            self._abs_root_path,
            'data_governance/library/data/custom_govsheet_schema.json'
        )
        custom_govsheet_schema = {}
        try:
            if os.path.isfile(custom_govsheet_schema_path):
                with open(custom_govsheet_schema_path, 'r') as f:
                    custom_govsheet_schema = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            custom_govsheet_schema = {}
        return custom_govsheet_schema


CustomGovSheet(os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

notebook_name = 'governance_sheet.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()